In [1]:
!pip install pandas
!pip install numpy
!pip install dash


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("./spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
datapie = spacex_df[['Launch Site','class']]

In [5]:
# Create a dash application
app = dash.Dash(__name__)

In [6]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                  dcc.Dropdown(id='site-dropdown',
                                                    options=[
                                                        {'label': 'All Sites', 'value': 'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-401'},
                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                     ],
                                                     value='ALL',
                                                     placeholder="Select a Launch Site here",
                                                     searchable=True
                                                     ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                # Function decorator to specify function input and output
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=min_payload, 
                                                max=max_payload, 
                                                step=100,
                                                marks={min_payload: str(min_payload), max_payload: str(max_payload)},
                                                value=[min_payload, max_payload]),
                               

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                        ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                                              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(datapie, values='class', 
        names='Launch Site', 
        title='Success percentage of each site')
        return fig
    else:
        dd=datapie[datapie['Launch Site'] == entered_site].groupby(['class']).size().reset_index(name='Count')
        dd['class']=dd['class'].replace({0:'Failure',1:'Success'})
        fig = px.pie(dd,values='Count',names='class',
        title='Success percentage of each site')
        return fig
                                        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                                              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site,entered_payload):
    if entered_site == 'ALL':
        filt_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= entered_payload[0]) & (spacex_df['Payload Mass (kg)'] <= entered_payload[1])]
        fig = px.scatter(filt_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',  
        title='Success vs Payload')
        return fig
    else:
        spacex_df_filt=spacex_df[spacex_df['Launch Site'] == entered_site]
        filt_df = spacex_df_filt[(spacex_df_filt['Payload Mass (kg)'] >= entered_payload[0]) & (spacex_df_filt['Payload Mass (kg)'] <= entered_payload[1])]
        fig = px.scatter(filt_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',  
        title='Success vs Payload')
        return fig

In [7]:
# Run the app
if __name__ == '__main__':
    app.run_server()